#Feature Extraction(skip if .csv made)

In [1]:
import librosa
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score

/tmp/ipykernel_1594/1929855782.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import pandas as pd

fp = pd.read_csv("file_path_raaga.csv")
fp.head()

,Unnamed: 0,filepath,mbid,raagas
0,13,Ashwath Narayanan at Arkay by Ashwath Narayana...,ffe5da02-8a71-4933-b414-62ca298dc777,purna chandrika
1,14,Ashwath Narayanan at Arkay by Ashwath Narayana...,a724bc1d-0eaa-44d1-ae2a-401c6ae62865,surati
2,15,Ashwath Narayanan at Arkay by Ashwath Narayana...,455c858a-da98-4094-a41d-41d0154037f2,shuddha saveri
3,16,Ashwath Narayanan at Arkay by Ashwath Narayana...,662dbd9a-6858-4899-9369-8079c6c22935,mohanam
4,17,Ashwath Narayanan at Arkay by Ashwath Narayana...,ab8fff58-8359-4af8-bc2a-6d623d4ac48b,thodi


In [3]:
from tqdm import tqdm  # Import tqdm for progress tracking

# Define the columns of the dataset DataFrame
columns = ['rmse', 'chroma_stft', 'spec_cent', 'spec_bw', 'rolloff', 'zcr', 'mfcc0', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'raga']
dataset = pd.DataFrame(columns=columns)
ragas = fp["raagas"].unique()

# Iterate over each raga and sample
for raga in tqdm(ragas, desc="Processing ragas"):
    sample = fp[fp["raagas"] == raga]
    
    # Iterate over each song in the sample
    for song in tqdm(sample["filepath"], desc="Processing songs", leave=False):
        sr = 0
        songname="carnatic/"+song+".mp3.mp3"
        y, sr = librosa.load(songname, mono=True)
        dur = librosa.get_duration(y=y, sr=sr)
        off = 0

        # Check duration and process accordingly
        if dur > 300:
            for i in tqdm(range(10), desc="Processing segments", leave=False):
                x, sr = librosa.load(songname, mono=True, offset=off, duration=30)
                rmse = librosa.feature.rms(y=x)[0]
                chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
                rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
                zcr = librosa.feature.zero_crossing_rate(x)
                mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=13)
                
                # Create data list and series
                data = [
                    np.mean(rmse), np.mean(chroma_stft), np.mean(spec_cent),
                    np.mean(spec_bw), np.mean(rolloff), np.mean(zcr),
                    np.mean(mfcc[0]), np.mean(mfcc[1]), np.mean(mfcc[2]),
                    np.mean(mfcc[3]), np.mean(mfcc[4]), np.mean(mfcc[5]),
                    np.mean(mfcc[6]), np.mean(mfcc[7]), np.mean(mfcc[8]),
                    np.mean(mfcc[9]), np.mean(mfcc[10]), np.mean(mfcc[11]),
                    np.mean(mfcc[12]), raga
                ]
                dataseries = pd.Series(data, index=dataset.columns)
                dataset = dataset._append(dataseries, ignore_index=True)

                # Update offset
                if i in range(0, 3):
                    off += 30
                if i in range(3, 7):
                    off = (dur / 10) * i
                if i in range(7, 10):
                    off = dur - ((10 - i) * 30)

        else:
            for i in tqdm(range(10), desc="Processing segments", leave=False):
                x, sr = librosa.load(songname, mono=True, offset=off, duration=(dur / 10))
                rmse = librosa.feature.rms(y=x)[0]
                chroma_stft = librosa.feature.chroma_stft(y=x, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=x, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=sr)
                rolloff = librosa.feature.spectral_rolloff(y=x, sr=sr)
                zcr = librosa.feature.zero_crossing_rate(x)
                mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=13)
                
                # Create data list and series
                data = [
                    np.mean(rmse), np.mean(chroma_stft), np.mean(spec_cent),
                    np.mean(spec_bw), np.mean(rolloff), np.mean(zcr),
                    np.mean(mfcc[0]), np.mean(mfcc[1]), np.mean(mfcc[2]),
                    np.mean(mfcc[3]), np.mean(mfcc[4]), np.mean(mfcc[5]),
                    np.mean(mfcc[6]), np.mean(mfcc[7]), np.mean(mfcc[8]),
                    np.mean(mfcc[9]), np.mean(mfcc[10]), np.mean(mfcc[11]),
                    np.mean(mfcc[12]), raga
                ]
                dataseries = pd.Series(data, index=dataset.columns)
                dataset = dataset._append(dataseries, ignore_index=True)
                
                # Update offset
                off = (dur / 10) * i

dataset.to_csv("raga-dataset-new.csv")


Processing ragas:   0%|          | 0/96 [00:00<?, ?it/s]
/tmp/ipykernel_4814/3989982456.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = dataset._append(dataseries, ignore_index=True)























Processing ragas:   1%|          | 1/96 [00:05<09:13,  5.83s/it]







































Processing ragas:   2%|▏         | 2/96 [00:15<12:34,  8.03s/it]











Processing ragas:   3%|▎         | 3/96 [00:18<08:49,  5.69s/it]










































Processing ragas:   4%|▍         | 4/96 [00:38<17:36, 11.48s/it]



















































































Processing ragas:   5%|▌         | 5/96 [01:16<32:01, 21.12s/it]











Processing ragas: 

In [2]:
data = pd.read_csv("raga-dataset-new.csv")
data.shape

(1840, 21)

In [3]:
data = data.drop(columns=['Unnamed: 0'])
data.head()

,rmse,chroma_stft,spec_cent,spec_bw,rolloff,zcr,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,raga
0,0.031511,0.317603,1708.003424,2073.255115,3612.529119,0.060923,-322.84384,109.459860,-2.044547,7.518317,-11.074581,5.313379,-22.433043,-11.289671,-8.702506,-8.501856,-9.121346,-2.449627,-11.605868,purna chandrika
1,0.031511,0.317603,1708.003424,2073.255115,3612.529119,0.060923,-322.84384,109.459860,-2.044547,7.518317,-11.074581,5.313379,-22.433043,-11.289671,-8.702506,-8.501856,-9.121346,-2.449627,-11.605868,purna chandrika
2,0.044276,0.338240,1690.241143,2038.245146,3691.999088,0.060042,-244.12135,118.934510,-5.504768,12.037164,-20.277803,9.172882,-22.398330,-12.889206,-6.194197,-10.408050,-11.628651,0.368445,-13.271334,purna chandrika
3,0.033655,0.283715,1707.162950,2062.905105,3553.485722,0.063138,-298.48642,111.781050,0.330066,12.399558,-15.397969,5.719809,-20.228254,-10.876356,-11.663798,-9.163310,-11.074247,-1.787798,-10.519550,purna chandrika
4,0.051188,0.287830,2224.945240,2284.799855,4762.157785,0.080556,-224.02823,87.893936,-7.011757,16.922640,-22.737362,11.527617,-34.319733,-6.622852,-3.180579,-10.729729,-11.729634,4.459793,-18.696102,purna chandrika


#Model Training

In [4]:
import os
import librosa
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.layers import Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

2024-04-20 22:48:45.629321: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-20 22:48:45.632314: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-20 22:48:45.660867: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 22:48:46.443852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
#Encoding the Labels
raga_list = data.iloc[:, -1]
encoder = LabelEncoder()
Y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

#ANN Model

In [6]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(96, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

/home/sypher/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-04-20 22:48:48.340937: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-20 22:48:48.341264: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 96)             │         6,240 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,512 (205.12 KB)

 Trainable params: 52,512 (205.12 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
classifier = model.fit(X_train, y_train,batch_size=128, epochs=500, validation_data=(X_test,y_test),callbacks=[checkpoint])

Epoch 1/500


 1/11 ━━━━━━━━━━━━━━━━━━━━ 8s 816ms/step - accuracy: 0.0000e+00 - loss: 4.5293
Epoch 1: val_loss improved from inf to 4.49336, saving model to best_model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0137 - loss: 4.5404 - val_accuracy: 0.0283 - val_loss: 4.4934
Epoch 2/500
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0391 - loss: 4.4491
Epoch 2: val_loss improved from 4.49336 to 4.44921, saving model to best_model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0428 - loss: 4.4196 - val_accuracy: 0.0283 - val_loss: 4.4492
Epoch 3/500
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0391 - loss: 4.3149
Epoch 3: val_loss improved from 4.44921 to 4.39591, saving model to best_model.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0391 - loss: 4.3231 - val_accuracy: 0.0391 - val_loss: 4.3959
Epoch 4/500
 1/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0469 - loss: 4.2641
Epoch 4: val_loss improved from 4.39591 to 4.32872, saving model t

In [8]:
y_pred = model.predict(X)
y_pred_classes = np.argmax(y_pred, axis=1)
total_accuracy = accuracy_score(Y, y_pred_classes)
print(f"Total accuracy on the entire dataset (X and Y): {total_accuracy:.4f}")

 1/58 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step

58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Total accuracy on the entire dataset (X and Y): 0.8707


LSTM Model

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
input_shape = (X_train.shape[1], X_train.shape[2])
model = Sequential()
adam1 = Adam(learning_rate=0.01)
model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.25, return_sequences=True,input_shape=input_shape))
model.add(LSTM(units=64,  dropout=0.05, recurrent_dropout=0.25, return_sequences=True))
model.add(Flatten())
model.add(Dense(units=96, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer=adam1, metrics=["accuracy"],)
model.summary()

/home/sypher/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 128)         │        75,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 96)             │         6,240 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,424 (513.38 KB)

 Trainable params: 131,424 (513.38 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
batch_size = 35 # num of training examples per minibatch
num_epochs =300
classify2 = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(X_test,y_test),
)

Epoch 1/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.0593 - loss: 4.3964 - val_accuracy: 0.0804 - val_loss: 3.6562
Epoch 2/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1196 - loss: 3.3883 - val_accuracy: 0.1370 - val_loss: 3.0103
Epoch 3/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2502 - loss: 2.6051 - val_accuracy: 0.2196 - val_loss: 2.6313
Epoch 4/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3430 - loss: 2.2224 - val_accuracy: 0.2891 - val_loss: 2.4163
Epoch 5/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4135 - loss: 1.9315 - val_accuracy: 0.2826 - val_loss: 2.3006
Epoch 6/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4475 - loss: 1.7615 - val_accuracy: 0.3413 - val_loss: 2.1628
Epoch 7/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4629 - loss: 1.6597 - val_accuracy: 0.3609 - val_loss: 2.0838
Epoch 8/300
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5148 - loss: 1.5178 - val_accuracy: 0.3717 - 

In [11]:
X_reshaped = np.reshape(X, (X.shape[0], 1, X.shape[1]))
y_pred = model.predict(X_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
total_accuracy = accuracy_score(Y, y_pred_classes)
print(f"Total accuracy on the entire dataset (X and Y): {total_accuracy:.4f}")

58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Total accuracy on the entire dataset (X and Y): 0.8652


#BiLSTM Model

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
input_shape = (X_train.shape[1], X_train.shape[2])
model = Sequential()
adam1 = Adam(learning_rate=0.001)
model.add(Bidirectional(LSTM(units=128, dropout=0.05, recurrent_dropout=0.25, return_sequences=True),input_shape=input_shape))
model.add(Bidirectional(LSTM(units=64,  dropout=0.05, recurrent_dropout=0.25, return_sequences=True)))
model.add(Flatten())
model.add(Dense(units=96, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer=adam1, metrics=["accuracy"],)
model.summary()

/home/sypher/.local/lib/python3.10/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 1, 256)         │       151,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 1, 128)         │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 96)             │        12,384 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 328,288 (1.25 MB)

 Trainable params: 328,288 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
batch_size = 35 # num of training examples per minibatch
num_epochs = 500
classify = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(X_test,y_test),
)

Epoch 1/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.0400 - loss: 4.5439 - val_accuracy: 0.0978 - val_loss: 4.4509
Epoch 2/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1059 - loss: 4.3410 - val_accuracy: 0.0848 - val_loss: 4.1350
Epoch 3/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0941 - loss: 3.9689 - val_accuracy: 0.0891 - val_loss: 3.8405
Epoch 4/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1024 - loss: 3.5989 - val_accuracy: 0.1000 - val_loss: 3.5701
Epoch 5/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1519 - loss: 3.2735 - val_accuracy: 0.1239 - val_loss: 3.3275
Epoch 6/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1638 - loss: 3.0637 - val_accuracy: 0.1500 - val_loss: 3.1288
Epoch 7/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2476 - loss: 2.8235 - val_accuracy: 0.1652 - val_loss: 2.9700
Epoch 8/500
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2633 - loss: 2.6749 - val_accuracy: 0.1804 - 

In [14]:
X_reshaped = np.reshape(X, (X.shape[0], 1, X.shape[1]))
y_pred = model.predict(X_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
total_accuracy = accuracy_score(Y, y_pred_classes)
print(f"Total accuracy on the entire dataset (X and Y): {total_accuracy:.4f}")

58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Total accuracy on the entire dataset (X and Y): 0.8848


# 1D Convolution Network

In [15]:
import os
import librosa
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
import tensorflow.keras.layers as layers
import keras
import tensorflow as tf
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.layers import Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [16]:
data = pd.read_csv('raga-dataset-new.csv')
# Dropping unneccesary columns
#Encoding the Labels
raga_list = data.iloc[:, -1]
encoder = LabelEncoder()
Y = encoder.fit_transform(raga_list)
#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
input_shape = (X_train.shape[1], X_train.shape[2])
X_train.shape[1], X_train.shape[2]

(1, 20)

In [17]:
model = tf.keras.Sequential()
model.add(layers.Conv1D(64, 3, activation='relu', padding='same', input_shape=input_shape))
model.add(layers.Conv1D(128, 3, activation='relu', padding='same'))
model.add(layers.Conv1D(256, 3, activation='sigmoid', padding='same'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(96, activation='softmax'))
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/home/sypher/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1, 64)          │         3,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 96)             │        12,384 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 172,448 (673.62 KB)

 Trainable params: 172,448 (673.62 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
classifier = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=1000,
    validation_data=(X_test, y_test),
)

Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.0339 - loss: 4.6059 - val_accuracy: 0.0435 - val_loss: 4.4288
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0365 - loss: 4.4571 - val_accuracy: 0.0380 - val_loss: 4.4046
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0397 - loss: 4.3903 - val_accuracy: 0.0380 - val_loss: 4.3869
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0431 - loss: 4.3570 - val_accuracy: 0.0380 - val_loss: 4.3646
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0450 - loss: 4.3442 - val_accuracy: 0.0471 - val_loss: 4.3314
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0370 - loss: 4.3261 - val_accuracy: 0.0707 - val_loss: 4.2783
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0419 - loss: 4.2367 - val_accuracy: 0.0598 - val_loss: 4.1998
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0508 - loss: 4.1529 - val_accuracy: 0

In [19]:
X_reshaped = np.reshape(X, (X.shape[0], 1, X.shape[1]))
y_pred = model.predict(X_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(Y, y_pred_classes)


print(f"accuracy: {accuracy:.4f}")

58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
accuracy: 0.9027
